In [ ]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import json
from transformers import AutoTokenizer, BertModel
import tqdm
import torch.nn as nn
import math

class OurDataset(Dataset):
    def __init__(self, data_file, labels_file):
        self.full_data = json.load(open(data_file))
        self.labels = torch.load(labels_file)

        self.tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
        self.model = BertModel.from_pretrained("bert-base-uncased")

    def __len__(self):
        return len(self.full_data)

    def __getitem__(self, idx):
        inputs = self.tokenizer(self.full_data[idx], return_tensors="pt")
        outputs = self.model(**inputs)
        last_hidden_states = outputs.last_hidden_state
        return last_hidden_states, self.labels[idx]

class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

def main():
    return TransformerClassifier()

class TransformerClassifier(nn.Module):

    def __init__(
        self,
        d_model=768,
        n_classes=3,
        nhead=4,
        dim_feedforward=256,
        num_layers=2,
        dropout=0.1,
        activation="relu",
        classifier_dropout=0.1,
    ):

        super().__init__()

        self.pos_encoder = PositionalEncoding(
            d_model=d_model,
            dropout=dropout,
            max_len=5000,
        )

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers,
        )
        self.head = nn.Sequential(
            nn.Linear(d_model, 256),
            nn.ReLU(),
            nn.Linear(256, n_classes),
            nn.Softmax(dim=1)
        )

        self.d_model = d_model
        self.dropout = nn.Dropout(p=classifier_dropout)

    def forward(self, x):
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        x = x.mean(dim=1)
        x = self.dropout(x)
        x = self.head(x)

        return x

In [ ]:
t = TransformerClassifier()
ds = OurDataset("full_data.json", "labels.torch")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

dl = DataLoader(ds)
# Assuming you have a DataLoader `data_loader` and the total dataset size `total_data_len`
# Example: DataLoader was created from a TensorDataset
# dataset = TensorDataset(X, y)
total_data_len = len(dl.dataset)

# Determine the sizes for the training and testing sets
train_size = int(0.7 * total_data_len)  # 70% for training
test_size = total_data_len - train_size  # Remaining 30% for testing

# Split the DataLoader into train and test DataLoaders
train_data, test_data = torch.utils.data.random_split(dl.dataset, [train_size, test_size])

# Create separate DataLoader objects for training and testing
train_loader = DataLoader(train_data, batch_size=dl.batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=dl.batch_size, shuffle=False)

# Now you have separate DataLoader objects for training and testing

In [ ]:
# Function to count parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Count parameters before training
params_before_training = count_parameters(t)
print(f"Number of parameters before training: {params_before_training}")


Number of parameters before training: 5716995


In [ ]:
# Define parameters to prune
parameters_to_prune = []

# Add linear layers in the head
parameters_to_prune.extend([(t.head[0], 'weight'), (t.head[2], 'weight')])

# Add linear layers in the transformer encoder
for layer in t.transformer_encoder.layers:
    # Add the linear layers within the feedforward part
    parameters_to_prune.extend([(layer.linear1, 'weight'), (layer.linear2, 'weight')])

    # Also, add the self-attention layers if desired
    parameters_to_prune.extend([(layer.self_attn, 'in_proj_weight'), (layer.self_attn.out_proj, 'weight')])
                            #(layer.self_attn, 'in_proj_bias'),

                            #(layer.self_attn.out_proj, 'bias')])

# Print the total number of parameters to prune
print("Total number of parameters to prune:", len(parameters_to_prune))

Total number of parameters to prune: 10


Train the model without pruning

In [ ]:
from torch.nn.utils import prune
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(t.parameters(), lr=0.00001, momentum=0.9)
loss = None
#for i in range(train_loader.__len__()):
dl = tqdm.tqdm(dl)
for i, data in enumerate(train_loader):
  #idx = random.randint(0,ds.__len__()-1)
  #x,y = train_loader.__getitem__(idx)
  #x,y = train_loader[idx]
  x, y = data
  y = y[0].long()
  outputs = t(x[0])
  loss = loss_fn(outputs, y)
  if i%10 == 0:
    print(i, loss.item())


  #optimizer.zero_grad()
  loss.backward()

  count_parameters(t)

  # Apply pruning
  '''prune.global_unstructured(
      parameters_to_prune,
      pruning_method=prune.L1Unstructured,
      amount=0.2,
  )'''

  '''# Remove pruned weights
  for layer in t.transformer_encoder.layers:
      prune.remove(layer.self_attn, 'in_proj_weight')
      prune.remove(layer.self_attn.out_proj, 'weight')
      #prune.remove(layer.self_attn, 'in_proj_bias')
      #prune.remove(layer.self_attn.out_proj, 'bias')

  prune.remove(t.head[0], 'weight')
  prune.remove(t.head[2], 'weight')
  print(count_parameters(t))'''

  optimizer.step()

0 1.0731910467147827
10 1.0920650959014893
20 1.1261510848999023
30 0.9553744792938232
40 0.8931958079338074
50 1.3433802127838135
60 0.6828944087028503
70 0.6261782050132751
80 0.5925154089927673
90 0.5771605968475342
100 0.5689197182655334
110 0.5607908368110657
120 0.5577723979949951
130 1.5496752262115479
140 0.5526040196418762
150 1.54966402053833
160 0.5517299771308899
170 0.5516777038574219
180 1.5513842105865479
190 0.5515176057815552
200 1.551396131515503
210 1.5514187812805176
220 0.5514523983001709
230 0.5514487028121948
240 1.5514435768127441
250 1.5514434576034546
260 1.5514438152313232
270 0.5514448881149292
280 1.5514445304870605
290 0.5514448881149292
300 0.5514447689056396
310 1.5514445304870605
320 1.5514447689056396
330 0.5514447093009949
340 1.5514447689056396
350 1.5514447689056396
360 1.5514447689056396
370 0.5514447093009949
380 0.5514447093009949
390 0.5514447093009949
400 0.5514447093009949
410 1.5514447689056396
420 1.5514447689056396
430 0.5514447093009949
44

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.type(torch.float32)
        y_pred = t(X_batch[0])
        y_pred_class = y_pred.argmax(dim=1)  # get the predicted class
        correct += (y_pred_class == y_batch.sum().item())
        total += y_batch.size(0)

acc = correct / total
print("Model accuracy: %.2f%%" % (acc*100))

Train a model with pruning

In [ ]:
t2 = TransformerClassifier()

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
# Define parameters to prune
parameters_to_prune = []

# Add linear layers in the head
parameters_to_prune.extend([(t2.head[0], 'weight'), (t2.head[2], 'weight')])

'''# Add linear layers in the transformer encoder
for layer in t2.transformer_encoder.layers:
    # Add the linear layers within the feedforward part
    parameters_to_prune.extend([(layer.linear1, 'weight'), (layer.linear2, 'weight')])

    # Also, add the self-attention layers if desired
    parameters_to_prune.extend([(layer.self_attn, 'in_proj_weight'), (layer.self_attn.out_proj, 'weight')])
                            #(layer.self_attn, 'in_proj_bias'),
                            #(layer.self_attn.out_proj, 'bias')])'''

# Print the total number of parameters to prune
print("Total number of parameters to prune:", len(parameters_to_prune))

Total number of parameters to prune: 2


In [ ]:
print(parameters_to_prune)

[(Linear(in_features=768, out_features=256, bias=True), 'weight'), (Linear(in_features=256, out_features=3, bias=True), 'weight')]


In [ ]:

from torch.nn.utils import prune
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(t2.parameters(), lr=0.00001, momentum=0.9)
loss = None
#for i in range(train_loader.__len__()):
dl = tqdm.tqdm(dl)
for i, data in enumerate(train_loader):
  #idx = random.randint(0,ds.__len__()-1)
  #x,y = train_loader.__getitem__(idx)
  #x,y = train_loader[idx]
  x, y = data
  y = y[0].long()
  outputs = t2(x[0])
  loss = loss_fn(outputs, y)
  if i%10 == 0:
    print(i, loss.item())

  #optimizer.zero_grad()
  loss.backward()

  count_parameters(t)

  # Apply pruning
  prune.global_unstructured(
      parameters_to_prune,
      pruning_method=prune.L1Unstructured,
      amount=0.2,
  )

  # Remove pruned weights

  prune.remove(t2.head[0], 'weight')
  prune.remove(t2.head[2], 'weight')

  '''for layer in t2.transformer_encoder.layers:
      prune.remove(layer.self_attn, 'in_proj_weight')
      prune.remove(layer.self_attn.out_proj, 'weight')
      #prune.remove(layer.self_attn, 'in_proj_bias')
      #prune.remove(layer.self_attn.out_proj, 'bias')'''



  optimizer.step()


  0%|          | 0/7675 [00:00<?, ?it/s]

0 1.143286108970642
10 1.1519304513931274
20 1.0907596349716187
30 1.0150468349456787
40 0.9197108745574951
50 1.2523589134216309
60 1.334970474243164
70 1.449912428855896
80 0.6236763596534729
90 1.478684902191162
100 1.5342555046081543
110 0.5608100891113281
120 0.5574564337730408
130 1.546830654144287
140 1.5487277507781982
150 0.5520928502082825
160 0.5516519546508789
170 0.5515257716178894
180 0.5514727234840393
190 0.5514631271362305
200 0.5514519810676575
210 1.551435947418213
220 1.5514413118362427
230 1.5514438152313232
240 0.5514448881149292
250 0.5514447689056396
260 0.5514447093009949
270 1.5514447689056396
280 1.5514447689056396
290 0.5514447093009949
300 1.5514447689056396
310 1.5514447689056396
320 1.5514447689056396
330 1.5514447689056396
340 0.5514447093009949
350 0.5514447093009949
360 0.5514447093009949
370 0.5514447093009949
380 0.5514447093009949
390 1.5514447689056396
400 1.5514447689056396
410 0.5514447093009949
420 1.5514447689056396
430 1.5514447689056396
440 0

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.type(torch.float32)
        y_pred = t2(X_batch[0])
        y_pred_class = y_pred.argmax(dim=1)  # get the predicted class
        correct += (y_pred_class == y_batch.sum().item())
        total += y_batch.size(0)

acc = correct / total
print("Model accuracy: %.2f%%" % (acc*100))

Model accuracy: 64.22%
